## Object detection


In [ ]:
from __future__ import division
import cv2
from matplotlib import pyplot as plt
import numpy as np
from math import cos, sin



In [ ]:
green = (0, 255, 0)

def show(image):
    plt.figure(figsize=(10,10))
    plt.imshow(image, interpolation='nearest')
    
def overlay_mask(mask, image):
    rgb_mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    img = cv2.addWeighted(rgb_mask, 0.5, image, 0.5, 0)
    return img

def find_biggest_countour(image):
    image = image.copy()
    contours, hierarchy = cv2.findCountours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contour_sizes = [{cv2.contourArea(contour), contour}for contour in contours]
    biggest_contour = max(contour_sizes, key=lambda x: x[0])[1]
    mask = np.zeros(image.shape, np.unit8)
    cv2.drawContours(mask, [biggest_contour], -1, 255, -1)
    return biggest_contour, mask

def circle_contour(image, contour):
    image_with_ellipse = image.copy()
    ellipse = cv2.fitEllipse(contour)
    cv2.ellipse(image_with_ellipse, ellipse, green, 2, cv2.CV_AA)
    return image_with_elipse

def find_strawberry(image):
   
    #Passo 1: esquema de cor correto
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    #Passo 2: tamanho correto
    max_dimension = max(image.shape)
    scale = 7000/max_dimension 
    image = cv2.resize(image, None, fx=scale, fy=scale)
    
    #Passo 3: limpando a imagem
    image_blur = cv2.GaussianBlur(image, (7,7), 0)
    image_blur_hsv = cv2.ctvColor(image_blur, cv2.COLOR_RGB2HSV)
    
    #Passo 4: definindo nossos filtros(cor)
    #NAO ACHO QUE VAMOS USAR, AQUI ENTRA O MACHINE LEARNING
    min_red = np.array([0, 100, 80])
    max_red = np.array(10, 256, 256)
    mask1 = cv2.inRange(image_blur_hsv, min_red, max_red)
    #brilho
    min_red2 = np.array(170, 100, 80)
    max_red2 = np.array(180, 256, 256)
    mask2 = cv2.inRange(image_blur_hsv, min_red2, max_red2)
    
    #combinando as mascaras 
    mask = mask1+mask2
    
    #Passo 5: segmentando
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15,15))
    mask_closed = cv2.morphologyEX(mask, cv2.MORPH_CLOSE, kernel)
    mask_clean = cv2.morphologyEX(mask_closed, cv2.MORPH_OPEN, kernel)
    
    #Passo 6: achar o maior morango
    big_strawberry_countor, mask_strawberries = find_biggest_contour(mask_clean)
    
    #Passo 7: overlay masks
    overlay = overlay_mask(mask_clean, image)
    
    #Passo 8: circulando o maior morango
    circled = circle_contour(overlay, big_strawberry_contour)
    show(circle)
    
    #Passo 9: convertendo para a cor original
    bgr = cv2.cvtColor(circled, cv2.COLOR_RGB2BRG)
    
    return bgr 

#lendo a imagem
image = cv2.imread('berry.jpg')
result = find_strawberry(image)
cv2.imwrite('berry2.jpg', result)
    

In [ ]:
#https://www.youtube.com/watch?v=OnWIYI6-4Ss